In [ ]:
#Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys,time,random,warnings
warnings.simplefilter("ignore", UserWarning)
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
import utils

# Causalml
import causalml
print(causalml.__version__)

from causalml.dataset import synthetic_data
from causalml.metrics.visualize import *
from causalml.propensity import calibrate

import logging
logger = logging.getLogger('causalml')
logger.setLevel(logging.DEBUG)
plt.style.use('fivethirtyeight')

# Benchmark
from xgboost import XGBRegressor
from causalml.inference.meta import XGBTRegressor, MLPTRegressor, LRSRegressor
from causalml.inference.meta import BaseSRegressor, BaseTRegressor, TMLELearner

# Causal net
import causal_nets
from causal_nets import causal_net_estimate

# Dragonnet
from causalml.inference.nn import DragonNet

# CEVAE 
from causalml.inference.nn import CEVAE
#import torch
from causalml.propensity import ElasticNetPropensityModel
from causalml.metrics import *
from causalml.dataset import simulate_hidden_confounder

# Tuning
#from bayes_opt import BayesianOptimization #Cannot work
from sklearn.model_selection import GridSearchCV

# Self-defined functions and codes that are omitted because of long running time
# Some configuration of the plots we will create later
%matplotlib inline  
plt.rcParams["figure.figsize"] = (12,6)


# APA_M1_data is an open repository for the store of the results or functions that we used in this assignment

url = 'https://raw.githubusercontent.com/Aubreyldy/APA_M1_data/main/Hillstrom.csv'
test = pd.read_csv(url) 

test.head()

# Query some properties of the known data

print('Dimensionality of the known data is {}'.format(test.shape))  # .shape returns a tupel
print('The known data set has {} cases.'.format(test.shape[0]))     # we can also index the elements of that tupel
print('The total number of elements is {}.'.format(test.size))

# Conversion as target
df = test.drop(['history_segment','visit','spend'], axis=1)

# One-hot-encoding except for "segment"
df = pd.get_dummies(df, columns = ['zip_code','channel'],
                     drop_first=True, dtype=np.int64)

# conversion in different segments
df.groupby(['conversion','segment']).size().unstack(fill_value=0)

np.random.randn(1000, 3)

df[['conversion','segment']]

#df.loc[((df['segment'] == 'Mens E-Mail')
df[['conversion','segment']].hist(by='segment',bins = 2,legend=True, stacked=True)

labels = ['G1', 'G2', 'G3', 'G4', 'G5']
men_means = [20, 35, 30, 35, 27]
women_means = [25, 32, 34, 20, 25]
men_std = [2, 3, 4, 1, 2]
women_std = [3, 5, 2, 3, 3]
width = 0.35       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots()

ax.bar(labels, men_means, width, yerr=men_std, label='Men')
ax.bar(labels, women_means, width, yerr=women_std, bottom=men_means,
       label='Women')

ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.legend()

plt.show()

# Mix the two pairwise dataset
df.loc[((df['segment'] == 'Mens E-Mail') | (df['segment'] == 'Womens E-Mail')),['treatment']]=1
df.loc[(df['segment'] == 'No E-Mail'),['treatment']]=0

# Before we examine sub-groups, it is meaningful to have a look at the overall ratio of treatment or not
df['treatment'].value_counts()

df.groupby(['conversion','treatment']).size().unstack(fill_value=0)

#split customers into 4 groups
df.loc[(df['treatment']==0) & (df['conversion']==1), 'response'] = 'CR'
df.loc[(df['treatment']==0) & (df['conversion']==0), 'response'] = 'CN'
df.loc[(df['treatment']==1) & (df['conversion']==1), 'response'] = 'TR'
df.loc[(df['treatment']==1) & (df['conversion']==0), 'response'] = 'TN'
df['response'] = df['response'].astype('str')

pie = df.response.value_counts(normalize=True)
print(pie)

plt.rcParams['font.size'] = 10.0
plt.pie(pie, 
        autopct='%.2f%%', 
        labels=('TN','CN','TR','CR'),
        pctdistance=0.6,
        explode=(0,0,0.65,0.2),
        radius=1)
plt.legend()
plt.show()

plt.figure(figsize=(20,10))  # enlarge the figure

# We create one histogram for each numeric variable and illustrate how to set the number of bins
df.hist(bins=20);
plt.tight_layout()

corr= df.corr()
f,ax = plt.subplots(figsize=(18, 15))
sns.heatmap(corr ,annot=True,linewidth=.5,fmt='1f');

# Save the space and speed up modeling
df['recency'] = df['recency'].astype('float64')
df['mens'] = df['mens'].astype('float64')
df['womens'] = df['womens'].astype('float64')
df['newbie'] = df['newbie'].astype('float64') 
df['conversion'] = df['conversion'].astype('float64')
df['zip_code_Surburban'] = df['zip_code_Surburban'].astype('float64')
df['zip_code_Urban'] = df['zip_code_Urban'].astype('float64')
df['channel_Phone'] = df['channel_Phone'].astype('float64')
df['channel_Web'] = df['channel_Web'].astype('float64')
df['treatment'] = df['treatment'].astype('float64')

# Extract target variable and feature matrix 
X = df.drop(['conversion','segment','treatment','response'], axis=1)
Y = df[['conversion']]
T = df[['treatment']]

# Creating training and test dataset
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(
    X, T, Y, test_size=0.2, random_state=42) # copy from causal net
### replace name
print(X_train.shape, X_test.shape, T_train.shape, T_test.shape, Y_train.shape, Y_test.shape)

df.info()

# T-learner
learner_t = BaseTRegressor(learner=XGBRegressor(learning_rate = 0.0004))
learner_t.fit(X=X_train.values, treatment=np.ravel(T_train.values), y=np.ravel(Y_train.values))
cate_t = learner_t.predict(X=X_test.values, treatment=np.ravel(T_test.values), y=np.ravel(Y_test.values)).flatten()
print(cate_t)

# S-learner
learner_s = BaseSRegressor(learner=XGBRegressor(learning_rate = 0.0004))
learner_s.fit(X=X_train.values, treatment=np.ravel(T_train.values), y=np.ravel(Y_train.values))
cate_s = learner_s.predict(X=X_test.values, treatment=np.ravel(T_test.values), y=np.ravel(Y_test.values)).flatten()
print(cate_s)

# Getting causal estimates
tau_pred, mu0_pred, prob_t_pred, psi_0, psi_1, history, history_ps = causal_net_estimate(
    [X_train, T_train, Y_train], [X_train, T_train, Y_train], [X_test, T_test, Y_test],
    hidden_layer_sizes=[60], dropout_rates=[0.5], batch_size=None, alpha=0.,
    r_par=0., optimizer='Adam', learning_rate=0.0003,
    max_epochs_without_change=30, max_nepochs=10000, seed=123, estimate_ps=False)

## Train the neural network: Dragonnet Model
# Take a little longer tima compared to Causal net

dragon = DragonNet(neurons_per_layer=200, targeted_reg=True)

## Fit the model
dragon.fit(X_train, T_train, Y_train)

## Predict ITE
dragon_ite = dragon.predict(X_test, T_test, Y_test)
#dragon_ate = dragon_ite.mean()

## Reorder the columns of df by binary (Gausian distribution) and continous (Bernoulli distribution) variables 
# Reorder features with binary first and continuous after

df_cevae = df[["mens","womens","newbie","segment","conversion","zip_code_Surburban","zip_code_Urban","channel_Phone","channel_Web","treatment","recency","history"]]
df_cevae

## Extract target variable and features 
X = df_cevae.drop(['conversion','segment','treatment'], axis=1)
Y = df_cevae[['conversion']]
T = df_cevae[['treatment']]

## Transform columns (X, Y, T) into array 
T_array = T.loc[:,'treatment'].values
T_array

Y_array = Y.loc[:,'conversion'].values
Y_array

X_array = X.loc[:,["mens","womens","newbie","zip_code_Surburban","zip_code_Urban","channel_Phone","channel_Web","recency","history"]].values
X_array


## Create training and test dataset
itr, ite = train_test_split(np.arange(X_array.shape[0]), test_size=0.2, random_state=1)
X_train_c, T_train_c, Y_train_c = X_array[itr], T_array[itr], Y_array[itr]
X_test_c, T_test_c, Y_test_c = X_array[ite], T_array[ite], Y_array[ite]
print(X_train_c.shape, X_test_c.shape, T_train_c.shape, T_test_c.shape, Y_train_c.shape, Y_test_c.shape)

## Apply CEVAE MODEL - DEFAULT settings

'''cevae = CEVAE(outcome_dist="bernoulli", #not default     ##Outcome distribution: "bernoulli" , "exponential", "laplace", "normal", "studentt"
              latent_dim=20,                          ##Dimension of latent variable
              hidden_dim=200,                         ##Dimension of hidden layers of fully connected networks
              num_epochs=50,                         ##Number of training epochs
              batch_size=100,                        ##Batch size
              learning_rate=0.001,                    ##the final learning rate will be learning_rate * learning_rate_decay
              learning_rate_decay=0.01, 
              num_layers=3) #not 2                    ##Number of hidden layers in fully connected networks

## fit 
losses = cevae.fit(X=torch.tensor(X_train_c, dtype=torch.float),
                   treatment=torch.tensor(T_train_c, dtype=torch.float),
                   y=torch.tensor(Y_train_c, dtype=torch.float))

## predict
cevae_ite = cevae.predict(X_test_c, T_test_c, Y_test_c)'''

# Because it took one hour to get the result, we store the data from prediction and import it directly

# Read the result
data_url1 = 'https://raw.githubusercontent.com/Aubreyldy/APA_M1_data/main/cevae_loss.csv'
data_url2 = 'https://raw.githubusercontent.com/Aubreyldy/APA_M1_data/main/cevae_ite.csv'

losses = pd.read_csv(data_url1)
cevae_ite = pd.read_csv(data_url2)

cevae_ite1 = cevae_ite['cevae_ite']

import sys
sys.path.append("/Users/aubrey/Documents/GitHub/APA_M1_data")  # Change the Path
import utils

###### T-learner
Tlearner = pd.DataFrame(columns=['lr','ate','auuc','qini'])

# Time records
start_time = time.time()
# Meta-parameter of one-layer-nn

# index for store
s=1

for h in np.linspace(0.0002,0.001,num=5):
    Params = {'1': h}
    ate_1, auuc_1, qini_1= cv_cn("T_learner",X_train, X_test,
                                 T_train, T_test, Y_train, Y_test,Params)
    Tlearner.loc[s, ['lr']] = h
    Tlearner.loc[s, ['ate']] = ate_1
    Tlearner.loc[s, ['auuc']] = auuc_1
    Tlearner.loc[s, ['qini']] = qini_1
    s=s+1
        
print("--- %s seconds ---" % (time.time() - start_time))


###### S-learner
Slearner = pd.DataFrame(columns=['lr','ate','auuc','qini'])
start_time = time.time()
# Meta-parameter of one-layer-nn

# index for store
s=1

for h in np.linspace(0.0002,0.001,num=5):
    Params = {'1': h}
    ate_2, auuc_2, qini_2= cv_cn("S_learner",X_train, X_test,
                                 T_train, T_test, Y_train, Y_test,Params)
    Slearner.loc[s, ['lr']] = h
    Slearner.loc[s, ['ate']] = ate_2
    Slearner.loc[s, ['auuc']] = auuc_2
    Slearner.loc[s, ['qini']] = qini_2
    s=s+1
        
print("--- %s seconds ---" % (time.time() - start_time))

# Read the result
data_url = 'https://raw.githubusercontent.com/Aubreyldy/APA_M1_data/main/ATE_AUUC_hls.csv'
ATE_AUUC_hls = pd.read_csv(data_url)
#ATE_AUUC_hls = pd.read_csv('.../ATE_AUUC_hls.csv',) 
ATE_AUUC_hls = ATE_AUUC_hls.drop(ATE_AUUC_hls.columns[0], axis=1)

# Plot Function
utils.ourplot(range(10,41), ATE_AUUC_hls,ATE_AUUC_hls, y11='ATE_hls',y12='ATE_hls', y21='AUUC_hls',y22='AUUC_hls',
        x='Numbers of Neurons in the Second Hidden Layer',
        title='ATE and AUUC via Different Numbers of Neurons in the Second Hidden Layer',num=1)

# Select the Hidden Layer Size with Best AUUC
A = np.array(ATE_AUUC_hls.AUUC_hls==max(ATE_AUUC_hls.AUUC_hls))
print("The maximum of AUUC is: ", max(ATE_AUUC_hls.AUUC_hls))
print("The corresponding QINI is: ", ATE_AUUC_hls.QINI_hls[np.where(A)[0].item()])
print("The corresponding ATE is: ", ATE_AUUC_hls.ATE_hls[np.where(A)[0].item()])

# Read the result
ATE_AUUC_dr = pd.read_csv('.../ATE_AUUC_dr.csv',) 
ATE_AUUC_dr = ATE_AUUC_dr.drop(ATE_AUUC_dr.columns[0], axis=1)

utils.ourplot(np.linspace(0,0.9,num=10), ATE_AUUC_dr,ATE_AUUC_dr,y11='ATE_dr', y12='ATE_dr', y21='AUUC_dr',y22='AUUC_dr',
        x='Droupout Rates',
        title='ATE and AUUC via Different Droupout Rates',num=1)

# Select the Dropout Rate with Best AUC
A = np.array(ATE_AUUC_dr.AUUC_dr==max(ATE_AUUC_dr.AUUC_dr))
print("The maximum of AUUC is: ", max(ATE_AUUC_dr.AUUC_dr))
print("The corresponding QINI is: ", ATE_AUUC_dr.QINI_dr[np.where(A)[0].item()])
print("The corresponding ATE is: ", ATE_AUUC_dr.ATE_dr[np.where(A)[0].item()])

# Read the result
ATE_AUUC_hln = pd.read_csv('.../ATE_AUUC_hln.csv',) 
ATE_AUUC_hln = ATE_AUUC_hln.drop(ATE_AUUC_hln.columns[0], axis=1)

utils.ourplot(range(10,41),ATE_AUUC_hls,ATE_AUUC_hln,y11='ATE_hls',y12='ATE_hln',y21='AUUC_hls',y22='AUUC_hln',
       x='Numbers of Neurons in the Second Hidden Layer: i',
       title='Comparison bewteen different pools',num=2,nn=31,lab='hidden_layer_size=[30]')

# Read the result
ATE_AUUC_2 = pd.read_csv('.../ATE_AUUC_2.csv',) 
ATE_AUUC_2 = ATE_AUUC_2.drop(ATE_AUUC_2.columns[0], axis=1)

ATE_AUUC_2.columns = ['hidden layer size','dropout rate','learning rate','ATE','AUUC']

%matplotlib inline  
plt.rcParams["figure.figsize"] = (20,10)
sns.lineplot(data=ATE_AUUC_2, x="dropout rate", y="AUUC", hue="hidden layer size", style="learning rate")

# Read the result
dragon2 = pd.read_csv('.../dragon1.csv',) 
dragon2 = dragon2.drop(dragon2.columns[0], axis=1)

# Plot
%matplotlib inline  
plt.rcParams["figure.figsize"] = (20,10)
sns.lineplot(data=dragon2, x="r", y="auuc", hue="nrp", style="lr")

# The Best Pool from Above
dragon2[dragon2['auuc']==max(dragon2['auuc'])]

alpha=0.7
bins=30
plt.figure(figsize=(6,4))

plt.hist(cate_t, alpha=alpha, bins=bins, label='T-learner')
plt.vlines(cate_s[0], 0, plt.axes().get_ylim()[1], label='S-lbearner',
           linestyles='dotted', colors='green', linewidth=2)
plt.title('Distribution of CATE Predictions by Benchmark Models')
plt.xlabel('Individual Treatment Effect (ITE/CATE)')
plt.ylabel('# of Samples')
_=plt.legend()
plt.tight_layout()

alpha=0.7
bins=30
plt.figure(figsize=(12,4))

plt.subplot(1, 3, 1)
plt.hist(tau_pred, alpha=alpha, bins=bins, label='Causal net')
plt.xlabel('Individual Treatment Effect (ITE/CATE)')
plt.ylabel('# of Samples')
_=plt.legend()

plt.subplot(1, 3, 2)
plt.hist((dragon_ite[:,1]-dragon_ite[:,0]), bins=bins, alpha=alpha,  label='Dragonnet')
plt.xlabel('Individual Treatment Effect (ITE/CATE)')
plt.ylabel('# of Samples')
_=plt.legend()

plt.subplot(1, 3, 3)
plt.hist(cevae_ite1, alpha=alpha,  label='CEVAE')
plt.xlabel('Individual Treatment Effect (ITE/CATE)')
plt.ylabel('# of Samples')
_=plt.legend()

plt.suptitle('Distribution of CATE Predictions by Different Methods')
plt.tight_layout()

dft = pd.DataFrame({'y': np.ravel(Y_test), 'w': np.ravel(T_test),
                    'T-Learner': cate_t.flatten(),'S-Learner': cate_s.flatten(),
                    'Causal net': tau_pred.flatten(),
                   'Dragonnet': (dragon_ite[:,1]-dragon_ite[:,0]).flatten(),
                   'CEVAE': cevae_ite1})
plot(dft, outcome_col='y', treatment_col='w')
